<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#file-conversion" data-toc-modified-id="file-conversion-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>file conversion</a></span></li><li><span><a href="#process-files" data-toc-modified-id="process-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>process files</a></span></li></ul></div>

In [1]:
# import packages for conversion
import os
os.environ['LIBROSA_CACHE_DIR'] = '/tmp/librosa_cache'
import librosa
import numpy as np

from pathlib import Path
from pydub import AudioSegment
from pydub.silence import split_on_silence

from pydub.utils import which
import librosa 
import soundfile as sf
from multiprocessing import Pool, cpu_count
AudioSegment.converter = which("ffmpeg")

In [2]:
librosa.cache.clear()

In [3]:
data= '/Users/badmin/DSC/birdcall_project/trial/'

In [44]:
os.getcwd()

'/Users/badmin/DSC/birdcall_project'


# file conversion 

In [5]:
from tqdm import tqdm

In [6]:
#data parallelism
num_workers = cpu_count()

In [7]:
files = librosa.util.find_files(data, ext=['wav']) 
filesArray = np.asarray(files)
print(len(filesArray))

3200


In [8]:
files

['/Users/badmin/DSC/birdcall_project/trial/XC102968.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC102969.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC102972.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC104511.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC104512.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC104514.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC104527.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC104528.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC108499.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC108501.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109028.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109034.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109035.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109299.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109300.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109301.wav',
 '/Users/badmin/DSC/birdcall_project/trial/XC109305.wav',
 '/Users/badmi

In [9]:
listfiles = list(os.listdir(data))

In [10]:
listfiles

['XC255273.wav',
 'XC132250.wav',
 'XC31090.wav',
 'XC161634.wav',
 'XC319723.wav',
 'XC125076.wav',
 'XC288984.wav',
 'XC131956.wav',
 'XC256975.wav',
 'XC204955.wav',
 'XC400968.wav',
 'XC179912.wav',
 'XC156729.wav',
 'XC122916.wav',
 'XC155234.wav',
 'XC164566.wav',
 'XC189808.wav',
 'XC163209.wav',
 'XC406666.wav',
 'XC466192.wav',
 'XC293630.wav',
 'XC211633.wav',
 'XC161422.wav',
 'XC292274.wav',
 'XC283379.wav',
 'XC131567.wav',
 'XC278861.wav',
 'XC169081.wav',
 'XC283386.wav',
 'XC250860.wav',
 'XC314950.wav',
 'XC369987.wav',
 'XC263266.wav',
 'XC144107.wav',
 'XC294188.wav',
 'XC146076.wav',
 'XC423357.wav',
 'XC348961.wav',
 'XC293817.wav',
 'XC305875.wav',
 'XC478421.wav',
 'XC321690.wav',
 'XC235197.wav',
 'XC348140.wav',
 'XC122453.wav',
 'XC260196.wav',
 'XC316014.wav',
 'XC494258.wav',
 'XC149225.wav',
 'XC324059.wav',
 'XC371064.wav',
 'XC381383.wav',
 'XC453934.wav',
 'XC185161.wav',
 'XC182168.wav',
 'XC389091.wav',
 'XC196109.wav',
 'XC124963.wav',
 'XC499967.wav'

In [61]:
myaudio = AudioSegment.from_file(filesArray[88])
channel_count = myaudio.channels    #Get channels
sample_width = myaudio.sample_width #Get sample width
duration_in_sec = len(myaudio) / 1000#Length of audio in sec
sample_rate = myaudio.frame_rate

print( "sample_width=",sample_width )
print ("channel_count=", channel_count)
print ("duration_in_sec=", duration_in_sec )
print ("frame_rate=", sample_rate)

sample_width= 2
channel_count= 1
duration_in_sec= 50.704
frame_rate= 32000


In [12]:
# Define a function to normalize a chunk to a target amplitude.
def match_target_amplitude(aChunk, target_dBFS):
    ''' Normalize given audio chunk '''
    change_in_dBFS = target_dBFS - aChunk.dBFS
    return aChunk.apply_gain(change_in_dBFS)

def split(filepath):
    name = filepath.replace('/Users/badmin/DSC/birdcall_project/trial/', '')
    #AudioSegment.from_mp3("your_audio.mp3")
    sound = AudioSegment.from_wav(filepath)
    dBFS = sound.dBFS
    chunks = split_on_silence(sound, 
           min_silence_len = 500,
           silence_thresh = dBFS-2,
           keep_silence=300
     
    )
 
    output_chunks = AudioSegment.empty()
    for chunk in chunks: output_chunks += chunk
    output_chunks.export(f"{data}{name}.wav", format="wav")
    


# process files

In [13]:
for f in tqdm(filesArray):

    split(f)

100%|██████████| 3200/3200 [1:05:00<00:00,  1.22s/it]
